# Dynamic Programming – Sprint 4

### Integrantes:
- Arthur Felipe – RM: 553320  
- Eduardo Pires – RM: 556527  
- Luca Monteiro – RM: 556906  
- Leonardo Munhoz – RM: 556824  
- Davi Vieira – RM: 556798  

---

### 🧠 Resumo do Projeto:
Nas unidades de diagnóstico, o consumo diário de insumos (reagentes e descartáveis) nem sempre é registrado com precisão, dificultando o controle de estoque e a previsão de reposição.  
Com base nesse contexto, este projeto propõe uma **solução otimizada com Programação Dinâmica (Dynamic Programming)**, capaz de determinar **quando e quanto pedir de cada insumo** para minimizar custos de pedido e de manutenção de estoque.  

A solução utiliza algoritmos recursivos, memoizados e iterativos para calcular o **plano ótimo de pedidos**, considerando o **custo fixo de pedido** e o **custo de armazenagem diária por unidade**.


## Formulação do Problema

**Estados:**  
- `i` → dia atual (1 ≤ i ≤ N). Representa o início do próximo pedido.  
- O estado guarda o ponto de decisão para atender a demanda dos dias `i..N`.

**Decisões:**  
- Escolher o último dia `j ≥ i` que será coberto por um pedido feito no dia `i`.  
- Ou seja, quanto tempo o pedido feito hoje deve cobrir no futuro.

**Função de Transição:**  
\[
dp[i] = \min_{j \geq i} ( Custo\_Pedido(i,j) + dp[j+1] )
\]

**Função Objetivo:**  
Minimizar o custo total composto por:
- Custo fixo de pedido `K`
- Custo de manutenção `h` proporcional às unidades estocadas e dias em estoque.

\[
Custo\_Pedido(i,j) = K + \sum_{t=i}^{j} (demanda_t \times (t-i) \times h)
\]


In [2]:
# ===========================================================
# – Desenvolvimento das versões Recursiva, Memoizada e Iterativa
# ===========================================================

from pprint import pprint
from functools import lru_cache

# Exemplo de demanda (10 dias)
demand = [8, 6, 10, 4, 7, 9, 3, 5, 11, 6]
N = len(demand)

# Parâmetros do modelo
K = 50    # custo fixo de pedido
h = 1.5   # custo de manutenção por unidade/dia

# Soma acumulada para facilitar cálculos
cum = [0] * (N + 1)
for i in range(1, N + 1):
    cum[i] = cum[i - 1] + demand[i - 1]

def demand_sum(i, j):
    return cum[j] - cum[i - 1]

def ordering_cost(start, end):
    total = K
    for t in range(start, end + 1):
        units = demand[t - 1]
        days_held = t - start
        total += units * days_held * h
    return total

# ---------- Versão Recursiva ----------
def dp_recursive(pos):
    if pos > N:
        return 0, []
    best_cost = float("inf")
    best_schedule = None
    for j in range(pos, N + 1):
        cost_here = ordering_cost(pos, j)
        rec_cost, rec_sched = dp_recursive(j + 1)
        total_cost = cost_here + rec_cost
        if total_cost < best_cost:
            best_cost = total_cost
            best_schedule = [(pos, j, demand_sum(pos, j))] + rec_sched
    return best_cost, best_schedule

# ---------- Versão Memoizada (Top-Down) ----------
@lru_cache(None)
def dp_memo_cost(pos):
    if pos > N:
        return 0
    best = float("inf")
    for j in range(pos, N + 1):
        best = min(best, ordering_cost(pos, j) + dp_memo_cost(j + 1))
    return best

def dp_memo_schedule():
    pos = 1
    schedule = []
    while pos <= N:
        best_j = None
        best = float("inf")
        for j in range(pos, N + 1):
            cand = ordering_cost(pos, j) + dp_memo_cost(j + 1)
            if cand < best:
                best = cand
                best_j = j
        schedule.append((pos, best_j, demand_sum(pos, best_j)))
        pos = best_j + 1
    return best, schedule

# ---------- Versão Iterativa (Bottom-Up) ----------
def dp_bottom_up():
    dp = [0.0] * (N + 2)
    choice = [None] * (N + 2)
    for i in range(N, 0, -1):
        best = float("inf")
        best_j = None
        for j in range(i, N + 1):
            cost = ordering_cost(i, j) + dp[j + 1]
            if cost < best:
                best = cost
                best_j = j
        dp[i] = best
        choice[i] = best_j
    schedule = []
    pos = 1
    while pos <= N:
        j = choice[pos]
        schedule.append((pos, j, demand_sum(pos, j)))
        pos = j + 1
    return dp[1], schedule, dp, choice


In [3]:
# ===========================================================
# – Verificação: as três versões produzem o mesmo resultado?
# ===========================================================

print("Demanda diária (dias 1..N):", demand)
print(f"Parâmetros: Custo fixo K={K}, Custo manutenção h={h}\n")

# Executa os métodos
rec_cost, rec_sched = dp_recursive(1)
memo_cost = dp_memo_cost(1)
memo_best, memo_sched = dp_memo_schedule()
bottom_cost, bottom_sched, dp_table, choice = dp_bottom_up()

print("Recursiva: ", rec_cost)
pprint(rec_sched)
print("\nMemoizada: ", memo_cost)
pprint(memo_sched)
print("\nBottom-Up: ", bottom_cost)
pprint(bottom_sched)

# Confirma que são iguais
assert abs(rec_cost - memo_cost) < 1e-6
assert abs(rec_cost - bottom_cost) < 1e-6
print("\n✅ Todas as versões produzem o mesmo custo mínimo.\n")

print("Plano ótimo de pedidos:")
for start, end, qty in bottom_sched:
    print(f"Pedido no dia {start} cobre dias {start}..{end} -> quantidade = {qty}")


Demanda diária (dias 1..N): [8, 6, 10, 4, 7, 9, 3, 5, 11, 6]
Parâmetros: Custo fixo K=50, Custo manutenção h=1.5

Recursiva:  261.0
[(1, 4, 28), (5, 8, 24), (9, 10, 17)]

Memoizada:  261.0
[(1, 4, 28), (5, 8, 24), (9, 10, 17)]

Bottom-Up:  261.0
[(1, 4, 28), (5, 8, 24), (9, 10, 17)]

✅ Todas as versões produzem o mesmo custo mínimo.

Plano ótimo de pedidos:
Pedido no dia 1 cobre dias 1..4 -> quantidade = 28
Pedido no dia 5 cobre dias 5..8 -> quantidade = 24
Pedido no dia 9 cobre dias 9..10 -> quantidade = 17


## 🧾 Relatório Final

A partir da modelagem do problema de controle de estoque como um caso de **Programação Dinâmica**, foi possível determinar o **plano ótimo de pedidos**, minimizando custos de manutenção e de reposição.

### Resultados:
- Custo total mínimo encontrado: **261.0**
- Plano ótimo:
  - Pedido no dia 1 cobre dias 1–4 (quantidade 28)
  - Pedido no dia 5 cobre dias 5–8 (quantidade 24)
  - Pedido no dia 9 cobre dias 9–10 (quantidade 17)

### Conclusão:
O modelo permite que as unidades de diagnóstico tenham maior visibilidade do consumo de insumos e possam planejar pedidos de forma eficiente, reduzindo desperdícios e garantindo reposição contínua.

As três implementações (recursiva, memoizada e iterativa) produzem **os mesmos resultados**, validando a consistência da solução.
